# ❗ 최종 선택 모델 훈련 요약

## 모델 구조
- Conv2D(64, 3x3, ReLU) + BatchNormalization + MaxPooling2D
- Conv2D(96, 3x3, ReLU) + BatchNormalization + MaxPooling2D
- Flatten
- Dense(32, ReLU) + L2 정규화(0.001) + Dropout(0.4)
- Dense(1, Sigmoid)

## 주요 하이퍼파라미터
- 이미지 입력 크기: 86x86  
- Batch Size: 32  
- Optimizer: Adam(learning_rate=0.0005)  
- loss: binary_crossentropy  
- Epoch 수: 20  
- Seed 고정: 42  

## 데이터 전처리 및 제너레이터 설정
- ImageDataGenerator를 사용한 데이터 증강  
  - rescale: 1./255
  - brightness_range: [0.7, 1.3]
  - rotation_range: 10
  - zoom_range: 0.1
  - width/height_shift_range: 0.1
  - fill_mode: 'nearest'
- grayscale 모드로 불러옴
- train/val split 비율: 80% / 20%

## 성능 평가
- Accuracy/Loss 그래프 시각화 및 저장  
- Classification Report 출력 및 저장  
- Confusion Matrix 시각화 및 저장  

---

# Import Library + Seed Setting

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

SEED=42
os.environ["PYTHONHASHSEED"]=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Using GPU

In [ ]:
gpus=tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Image Augmentation

In [ ]:
IMG_SIZE=86
BATCH_SIZE=32
DATA_PATH="../../data/processed"

datagen=ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.7, 1.3],
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode="nearest",
    horizontal_flip=False,
    validation_split=0.2
)

train_generator=datagen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_generator=datagen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

# Define Model (CNN)

In [ ]:
model=Sequential([
    Conv2D(64, 3, activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    BatchNormalization(),
    MaxPooling2D(2),

    Conv2D(96, 3, activation="relu"),
    BatchNormalization(),
    MaxPooling2D(2),

    Flatten(),
    Dense(32, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer=Adam(learning_rate=0.0005),
            loss="binary_crossentropy", metrics=["accuracy"])

# Model Training

In [ ]:
history=model.fit(train_generator, validation_data=val_generator,
                epochs=20, verbose=1)

# Accuracy/Loss Graph

In [ ]:

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.savefig("../../results/images/training_plot_final.png")
plt.show()

# Classification Report

In [ ]:
y_true=val_generator.classes
steps=val_generator.samples // val_generator.batch_size + 1
y_pred_prob=model.predict(val_generator, steps=steps, verbose=0)
y_pred=(y_pred_prob > 0.5).astype(int)

report=classification_report(y_true, y_pred, target_names=["closed", "open"])
print(report)
with open("../../results/reports/classification_report_final.txt", "w") as f:
    f.write(report)

# confusion matrix

In [ ]:
conf_matirx=confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matirx, annot=True, fmt="d", cmap="Reds",
            xticklabels=["closed", "open"], yticklabels=["closed", "open"])
plt.xlabel("Pred")
plt.ylabel("Real")
plt.tight_layout()
plt.savefig("../../results/images/confusion_matrix_final.png")
plt.show()

# Save Model

In [ ]:
os.makedirs("../../src/models/Model_final", exist_ok=True)
model.save("../../src/models/Model_final/model_final.keras")

In [ ]:
os.makedirs("../../webcam_app/model", exist_ok=True)
model.save("../../webcam_app/model/model_final.keras")